### Basic configuration

In [ ]:
api_host = "https://staging.networkperspective.io"
network_id = "ad8c3e20-5651-4e48-a17d-44fafd0262da"

### Login

Get user creadentials

In [ ]:
import getpass

# don't ask for username and password if they are already set
if 'username' not in locals():    
    username = input("Enter your username: ") 
    password = getpass.getpass('Enter your password: ')

Acquire access token

In [ ]:
from np_client import np_client

configuration = np_client.Configuration(
    host = api_host
)

with np_client.ApiClient(configuration) as api_client:
    token_api = np_client.TokenApi(api_client)
    try:    
        token = token_api.token_get_token(
            username=username,  
            password=password, 
            grant_type="password", 
            client_id="ApiClient"
        )
    except Exception as e:
        print("Exception when calling TokenApi->token_get_token: %s\n" % e)  
        raise e

# set the token in the configuration to use it in the next calls
configuration.access_token = token.access_token

# token should be valid for 1 hour
print("Token expiration: ", token.expires)

### Get available metrics and periods

In [ ]:
with np_client.ApiClient(configuration) as api_client:    
    report_api = np_client.ReportApi(api_client)
    metrics_export_params_query = np_client.MetricsExportParamsQuery() 
    metrics_export_params_query.network_id = network_id
    metrics_export_params_query.aggregate = np_client.AggregateEnum.MONTH
    try:
        # Export metric params
        export_metric_params = report_api.report_export_metric_params(metrics_export_params_query=metrics_export_params_query)                
    except Exception as e:
        print("Exception when calling ReportApi->report_export_metric_params: %s\n" % e)
        raise e

print("periods", export_metric_params.periods)
print("metrics", export_metric_params.metrics)

In [ ]:
last_period = export_metric_params.periods[-1]
all_metrics = export_metric_params.metrics

### Export direct metric values
These are metrics calculated for each team or employee group.

In [ ]:
with np_client.ApiClient(configuration) as api_client:    
    report_api = np_client.ReportApi(api_client)

    metrics_export_query = np_client.MetricsExportQuery()     
    metrics_export_query.network_id = network_id
    metrics_export_query.periods = [last_period]
    metrics_export_query.metrics = all_metrics
    
    try:        
        metrics = report_api.report_export_metric_values(metrics_export_query=metrics_export_query)        
    except Exception as e:
        print("Exception when calling ReportApi->report_export_metric_values: %s\n" % e)
        raise e

Convert to dataframe for easy export & analysis

In [ ]:
import pandas as pd

metrics_df = pd.DataFrame([m.to_dict() for m in metrics])
display(metrics_df.head())

### Read realted metrics
These are cooperation paths between groups, read first 1000 values.

In [ ]:
with np_client.ApiClient(configuration) as api_client:    
    report_api = np_client.ReportApi(api_client)

    metric_related_export_query = np_client.MetricRelatedExportQuery()     
    metric_related_export_query.network_id = network_id
    metric_related_export_query.metric = "CooperationPaths"
    metric_related_export_query.top = 1000
    
    try:        
        related_metrics = report_api.report_export_metric_related_values(metric_related_export_query=metric_related_export_query)        
    except Exception as e:
        print("Exception when calling ReportApi->report_export_metric_related_values: %s\n" % e)
        raise e

Convert to dataframe

In [ ]:
related_df = pd.DataFrame([m.to_dict() for m in related_metrics])
display(related_df.head())